In [1]:
%matplotlib widget

In [2]:
from pprint import pprint
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Observations

In [3]:
pd.set_option('display.max_columns', 60)

# TDS Catalog
* ADS entry: https://ui.adsabs.harvard.edu/abs/2013ApJ...766...60G/abstract
* Resoruces at https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJ/766/60#/browse

This script's objective is to parse the official TDS catalog files and get the GALEX field IDs.  
The IDs can be used as input for the UVVA pipeline.

In [4]:
# read catalog CSV file
# csv is derived from original catalog resource "table4.dat"
tds_labels = [
    "ID",
    "RAdeg",
    "DEdeg",
    "l_mlow",
    "mlow",
    "l_Dmmax",
    "Dmmax",
    "l_sigint",
    "sigint",
    "l_Sd",
    "Sd",
    "l_Sy",
    "Sy",
    "LC",
    "Mph",
    "rmag",
    "OT",
    "z",
    "X",
    "Class",
]
tds_srcs = pd.read_csv(
    f"table4.csv",
    sep=";",
    header=[0,1]
)
display(tds_srcs)

,recno,ID,RAJ2000,DEJ2000,l_mlow,mlow,l_Dmmax,Dmmax,l_sigint,sigint,l_Sd,Sd,l_Sy,Sy,LC,Mph,rmag,OT,z,X,Class
,,,deg,deg,,mag,,mag,,mag,,mag,,mag,,,mag,,,,Unnamed: 20_level_1
0,1,GROTH_MOS01-21,216.1622,54.0911,,22.54,,4.60,,1.04,,0.80,,1.13,V,pt,14.92,Mdw,,,Mdw
1,2,VVDS22H_MOS05-05,333.8326,-0.5491,,21.14,,4.47,,0.99,,0.90,,0.74,F,pt,21.24,QSO,,,CV
2,3,ELAISN1_MOS15-02,242.0397,54.3586,>,22.89,>,4.03,,1.37,,0.84,,2.93,V,pt,22.51,QSO,,,CV
3,4,ELAISN1_MOS15-09,242.3685,53.6738,>,23.03,>,4.02,,1.23,,0.01,,2.15,F,ext,21.05,,,,Galaxy Trans
4,5,GROTH_MOS07-09,212.5024,52.4153,>,23.10,>,3.61,,3.51,,0.09,,0.69,F,pt,17.45,Mdw,,,Mdw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,1074,VVDS22H_MOS05-00,333.4082,-0.7183,,18.06,,0.26,,0.04,,0.02,,0.10,V,pt,17.19,QSO,1.182,,QSO
1074,1075,COSMOS_MOS23-20,150.9247,2.0944,,19.20,,0.26,,0.05,,0.03,,0.14,V,ext,16.77,,,,AGN
1075,1076,GROTH_MOS04-15,212.2576,53.1997,,19.79,,0.24,,0.05,,0.12,,0.07,V,pt,16.36,RRL,,,RRL


In [5]:
# parse target names so astroquery can understand it
tds_target_names_raw = np.asarray(
    [
        [
            key.split("-")[0],
            key.split("-")[1],
        ]
        for key in tds_srcs["ID"].values.T.tolist()[0]
    ]
)
tds_target_names = np.unique(tds_target_names_raw[:,0]).tolist()

In [6]:
# get field IDs form astroquery
field_ids = list()
for name in tds_target_names:
    
    # get MAST entry by target name
    tt_coadd = Observations.query_criteria(
                    target_name=f"PS_{name}",
                    dataRights="PUBLIC",
                    instrument_name="GALEX",
                    filters="NUV",
                    dataproduct_type="image",
                )
    field_id = int(tt_coadd["obs_id"].data[0])
    print(f"PS_{name}", field_id)
    field_ids.append(field_id)
field_ids

PS_CDFS_MOS00 2593653509626789888
PS_CDFS_MOS01 2593723878370967552
PS_CDFS_MOS02 2593794247115145216
PS_CDFS_MOS03 2593864615859322880
PS_CDFS_MOS04 2593934984603500544
PS_CDFS_MOS05 2594005353347678208
PS_CDFS_MOS06 2594075722091855872
PS_COSMOS_MOS21 2604912508670050304
PS_COSMOS_MOS22 2604982877414227968
PS_COSMOS_MOS23 2605053246158405632
PS_COSMOS_MOS24 2605123614902583296
PS_COSMOS_MOS25 2605193983646760960
PS_COSMOS_MOS26 2605264352416104448
PS_COSMOS_MOS27 2605334721135116288
PS_ELAISN1_MOS10 2597312684324028416
PS_ELAISN1_MOS11 2597383053068206080
PS_ELAISN1_MOS12 2597453421812383744
PS_ELAISN1_MOS13 2597523790556561408
PS_ELAISN1_MOS14 2597594159300739072
PS_ELAISN1_MOS15 2597664528044916736
PS_ELAISN1_MOS16 2597734896789094400
PS_GROTH_MOS01 2607023571020546048
PS_GROTH_MOS02 2607093939764723712
PS_GROTH_MOS03 2607164308508901376
PS_GROTH_MOS04 2607234677253079040
PS_GROTH_MOS05 2607305045997256704
PS_GROTH_MOS06 2607375414741434368
PS_GROTH_MOS07 2607445783485612032
PS_VVD

[2593653509626789888,
 2593723878370967552,
 2593794247115145216,
 2593864615859322880,
 2593934984603500544,
 2594005353347678208,
 2594075722091855872,
 2604912508670050304,
 2604982877414227968,
 2605053246158405632,
 2605123614902583296,
 2605193983646760960,
 2605264352416104448,
 2605334721135116288,
 2597312684324028416,
 2597383053068206080,
 2597453421812383744,
 2597523790556561408,
 2597594159300739072,
 2597664528044916736,
 2597734896789094400,
 2607023571020546048,
 2607093939764723712,
 2607164308508901376,
 2607234677253079040,
 2607305045997256704,
 2607375414741434368,
 2607445783485612032,
 2597875634277449728,
 2597946003021627392,
 2598016371765805056,
 2598086740509982720,
 2598157109254160384,
 2598227477998338048,
 2598297846742515712,
 2592597978464124928,
 2592668347208302592,
 2592738715952480256,
 2592809084696657920,
 2592879453440835584,
 2592949822185013248,
 2593020190929190912]

In [7]:
# parse catalog data directly with astropy
with fits.open(f"table4.fit") as hdul:
    hdul.info()

tt_tds_srcs = Table.read(f"table4.fit", hdu="J_ApJ_766_60_table4")

Filename: table4.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      69   ()      
  1  J_ApJ_766_60_table4    1 BinTableHDU    110   1078R x 21C   [J, 16A, D, D, A, E, A, E, A, E, A, E, A, E, A, 3A, E, 3A, E, A, 12A]   


In [8]:
tt_tds_srcs

recno,ID,RAJ2000,DEJ2000,l_mlow,mlow,l_Dmmax,Dmmax,l_sigint,sigint,l_Sd,Sd,l_Sy,Sy,LC,Mph,rmag,OT,z,X,Class
,,deg,deg,,mag,,mag,,mag,,mag,,mag,,,mag,,,,
int32,bytes16,float64,float64,bytes1,float32,bytes1,float32,bytes1,float32,bytes1,float32,bytes1,float32,bytes1,bytes3,float32,bytes3,float32,bytes1,bytes12
1,GROTH_MOS01-21,216.1622,54.0911,,22.54,,4.60,,1.04,,0.80,,1.13,V,pt,14.92,Mdw,--,,Mdw
2,VVDS22H_MOS05-05,333.8326,-0.5491,,21.14,,4.47,,0.99,,0.90,,0.74,F,pt,21.24,QSO,--,,CV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,GROTH_MOS06-03,214.1765,52.3035,,18.54,,0.24,,0.08,,0.02,,0.23,V,pt,17.57,QSO,1.282,X,QSO
1078,CDFS_MOS05-34,52.5501,-26.9459,,18.23,,0.23,,0.04,,0.05,,0.07,V,ext,15.35,,--,,AGN


In [9]:
# lookup table mapping mcat flux aperture index to physical diameter
# http://www.galex.caltech.edu/wiki/Public:Documentation/Appendix_A.1
pd.DataFrame(
    data=np.asarray(
    [
        np.array([1,2,3,4,5,6,7]),
        np.array([2, 3, 5, 8, 12, 17, 23]),
        np.array(np.array([2, 3, 5, 8, 12, 17, 23]) * 1.5 /2)
    ]
).T,
    columns=[
        "X",
        "ap. diameter [pix]",
        "ap. radius [arcsec]"
    ]
)

,X,ap. diameter [pix],ap. radius [arcsec]
0,1.0,2.0,1.50
1,2.0,3.0,2.25
2,3.0,5.0,3.75
3,4.0,8.0,6.00
4,5.0,12.0,9.00
5,6.0,17.0,12.75
6,7.0,23.0,17.25
